## Setting up notebook

In [ ]:
# installing packages
!pip install pip==24.0
!pip install numpy==1.23.5
!pip install tensorboardX
!pip install subword-nmt
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.9 MB/s eta 0:00:00


In [ ]:
# importing packages
import numpy
import os
import tensorboardX
import sentencepiece as spm

In [ ]:
# mounting google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
source_code = 'eng'
target_code = 'nde'

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}')

# installing fairseq
#!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
!pip install --editable ./

/content/drive/MyDrive/Research/eng-to-nde/fairseq
Obtaining file:///content/drive/MyDrive/Research/eng-to-nde/fairseq
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached hydra_core-1.0.7-py3-none-any.whl.metadata (3.7 kB)
  Using cached omegaconf-2.0.6-py3-none-any.whl.metadata (3.0 kB)
  Using cached sacrebleu-2.4.3-py3-none-any.whl.metadata (51 kB)
  Using cached bitarray-2.9.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (34 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 23.4 MB/s eta 0:00:00
  Bu

## Tokenizing the data

In [ ]:
train_source_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data/train.{source_code}'
train_target_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data/train.{target_code}'
val_source_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data/val.{source_code}'
val_target_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data/val.{target_code}'
test_source_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data/test.{source_code}'
test_target_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data/test.{target_code}'

### BPE

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/{source_code}-to-{target_code}/bpe/data')

In [ ]:
# learn joint vocabulary
!subword-nmt learn-joint-bpe-and-vocab --input $train_source_path $train_target_path -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$source_code vocab.$target_code

100% 4000/4000 [00:44<00:00, 90.43it/s] 


In [ ]:
# applying tokenization to training, validation and test sets
!subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$source_code < $train_source_path > train.BPE.$source_code
!subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$target_code < $train_target_path > train.BPE.$target_code

!subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$source_code < $val_source_path > val.BPE.$source_code
!subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$target_code < $val_target_path > val.BPE.$target_code

!subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$source_code < $test_source_path > test.BPE.$source_code
!subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$target_code < $test_target_path > test.BPE.$target_code

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/{source_code}-to-{target_code}/bpe')

In [ ]:
# prepare dictionary to be used by our nmt model
!fairseq-preprocess -s eng -t nde \
--trainpref data/train.BPE \
--validpref data/val.BPE \
--testpref data/test.BPE \
--joined-dictionary \
--destdir data-bin \
--bpe 'subword_nmt' \
--thresholdsrc 1 \
--thresholdtgt 1 \
--workers 2

2024-10-14 09:54:32.206706: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 09:54:32.456521: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 09:54:32.525262: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-14 09:54:32.923628: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-14 09:54:35.330201: W tensorflow/compiler/tf2

### ULM

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulm/data')

In [ ]:
# learn joint vocabulary
spm.SentencePieceTrainer.train(f'--input={train_source_path},{train_target_path} --model_prefix=joint --model_type=unigram --vocab_size=4000 --character_coverage=1.0 --bos_id=0 --pad_id=1 --eos_id=2 --unk_id=3')

In [ ]:
# applying tokenization to training, validation and test sets
sp = spm.SentencePieceProcessor(model_file='joint.model')

with open(train_source_path, 'r') as rf, open(f'train.ULM.{source_code}', 'w') as wf:
    for line in rf:
        wf.write(' '.join(sp.encode(line, out_type=str)) + '\n')

with open(train_target_path, 'r') as rf, open(f'train.ULM.{target_code}', 'w') as wf:
    for line in rf:
        wf.write(' '.join(sp.encode(line, out_type=str)) + '\n')

with open(val_source_path, 'r') as rf, open(f'val.ULM.{source_code}', 'w') as wf:
    for line in rf:
        wf.write(' '.join(sp.encode(line, out_type=str)) + '\n')

with open(val_target_path, 'r') as rf, open(f'val.ULM.{target_code}', 'w') as wf:
    for line in rf:
        wf.write(' '.join(sp.encode(line, out_type=str)) + '\n')

with open(test_source_path, 'r') as rf, open(f'test.ULM.{source_code}', 'w') as wf:
    for line in rf:
        wf.write(' '.join(sp.encode(line, out_type=str)) + '\n')

with open(test_target_path, 'r') as rf, open(f'test.ULM.{target_code}', 'w') as wf:
    for line in rf:
        wf.write(' '.join(sp.encode(line, out_type=str)) + '\n')

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulm')

In [ ]:
# prepare dictionary to be used by our nmt model
!fairseq-preprocess -s eng -t nde \
--trainpref data/train.ULM \
--validpref data/val.ULM \
--testpref data/test.ULM \
--joined-dictionary \
--destdir data-bin \
--bpe 'sentencepiece' \
--thresholdsrc 1 \
--thresholdtgt 1 \
--workers 2

2024-10-14 09:59:41.451213: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 09:59:41.481394: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 09:59:41.490168: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-14 09:59:41.510953: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-14 09:59:43.451146: W tensorflow/compiler/tf2